After doing some reading, I think the recommended solution shown here is crap. I can't figure out how to avoid either the read or write timeouts. Also, BufferedRWPair is not supposed to be used with the same object, and attempts to close the same serial port twice.

In [1]:
import io
import serial

In [2]:
s = serial.Serial('/dev/ttyUSB2', timeout=1, baudrate=19200, rtscts=True)

In [3]:
sio = io.TextIOWrapper(io.BufferedRWPair(s, s, buffer_size=1), newline=None, line_buffering=True)

In [4]:
sio.write(unicode('*IDN?\r'))
%time sio.readline()

CPU times: user 1 ms, sys: 999 µs, total: 2 ms
Wall time: 2.05 s


u'h_Systems,SR830,s/n45571,ver1.07 \n'

In [5]:
sio.write(unicode('*IDN?\r'))
sio.flush()
%time sio.readline()

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 2.05 s


u'Stanford_Research_Systems,SR830,s/n45571,ver1.07 \n'

In [6]:
# Gross
sio.close()

SerialException: Attempting to use a port that is not open

Try the new lock-in code.

In [7]:
from equipment.srs import lockin

reload(lockin)

In [8]:
l = lockin.SR830('/dev/ttyUSB2')

In [9]:
%time l.identification

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 55.4 ms


('Stanford_Research_Systems', 'SR830', 's/n45571', 'ver1.07')

In [10]:
%timeit l.R

10 loops, best of 3: 38.2 ms per loop


In [11]:
# The SRS seems to terminate some (all?) commands with a space and carriage return:
int('0 \r')

0

In [12]:
%time l.state()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 529 ms


{'detection_harmonic': 1,
 'identification': ('Stanford_Research_Systems',
  'SR830',
  's/n45571',
  'ver1.07'),
 'input_configuration': 0,
 'input_coupling': 1,
 'input_notch_filter': 1,
 'input_shield_grounding': 0,
 'local': 0,
 'output_filter_slope': 1,
 'reference_frequency': 122.071,
 'reference_phase': -172.0,
 'reference_source': 0,
 'reference_trigger': 2,
 'reserve_mode': 2,
 'rms_voltage': 0.0290757,
 'sample_rate': 4,
 'sensitivity': 23,
 'sine_output_voltage': 0.87,
 'sync_filter': False,
 'time_constant': 8,
 'voltage_phase': -0.96}